In [1]:
from examples.util.getDataset import getDataset
from examples.util.plotting import plot_w
from examples.util.test_fns import *
from examples.util.testing import shrimp_v_random, growth
from examples.util.metrics import shrimp_test, make_l2_loss
from sparse_rf.util import *
from sparse_rf.modules import make_A, make_W
from sparse_rf.algs import shrimp, l2, l1, sindy
from functools import partial
import matplotlib.pyplot as plt
from math import ceil

In [2]:
r = 9
fn = partial(fn9, r=r)
q = 1
d = 100
m = 200
N = 1500
l = 1e-13

Xtr, Xte, Ytr, Yte = getDataset(fn, d=d, m=m)

trials = 10

scales = np.arange(1/q, 206, 5)

In [3]:
for scale_ in scales:
    svr_q = shrimp_v_random(trials, Xtr, Ytr, Xte, Yte, q, N, l, scale=np.sqrt(scale_))
    print("Variance: {}".format(scale_))
    print("Min l2: {}".format(svr_q.err_l2))
    print("SHRIMP: {}".format(svr_q.err_shrimp))
    print("Random pruning: {}".format(svr_q.err_r))
    print("----------------------------")

Variance: 1.0
Min l2: 158.5122370298729
SHRIMP: 68.94646022912866
Random pruning: 43.793673762453416
----------------------------
Variance: 6.0
Min l2: 70.97459917952662
SHRIMP: 41.30513417589621
Random pruning: 47.624079895027954
----------------------------
Variance: 11.0
Min l2: 52.69777631727578
SHRIMP: 23.567380780412034
Random pruning: 37.1440330190227
----------------------------
Variance: 16.0
Min l2: 46.55510497460143
SHRIMP: 28.63382713857074
Random pruning: 46.16084411801647
----------------------------
Variance: 21.0
Min l2: 43.390535685840334
SHRIMP: 8.645662154961887
Random pruning: 38.817543750914616
----------------------------
Variance: 26.0
Min l2: 40.38657311358002
SHRIMP: 0.9203102907717456
Random pruning: 42.07449217315097
----------------------------
Variance: 31.0
Min l2: 35.49252384956149
SHRIMP: 4.5350241731642935
Random pruning: 46.51236258423364
----------------------------
Variance: 36.0
Min l2: 36.64468975847349
SHRIMP: 5.495473546734926
Random pruning: 47.